In [1]:
! pip install adversarial-robustness-toolbox

     |████████████████████████████████| 1.1 MB 949 kB/s eta 0:00:01


# Module

In [1]:
#Import Library
import random
import numpy as np
import matplotlib.pyplot as plt
import time

# TensorFlow with Keras.
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Flatten, Conv2D
from tensorflow.keras.layers import MaxPooling2D, GlobalAveragePooling2D, Dropout
tf.compat.v1.disable_eager_execution()

### Dataset

In [2]:
# MNIST 데이터셋 준비

mnist = keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()   

# 각 데이터 크기 출력
print(f"x_train.shape: {x_train.shape}")
print(f"y_train.shape: {y_train.shape}")
print(f"x_test.shape: {x_test.shape}")
print(f"y_test.shape: {y_test.shape}")

x_train.shape: (60000, 28, 28)
y_train.shape: (60000,)
x_test.shape: (10000, 28, 28)
y_test.shape: (10000,)


# Preprocessing

In [3]:
# Normalization - x
print('최소값:',np.min(x_train), ' 최대값:',np.max(x_train))

x_train_norm = x_train.astype('float32')/255.0
x_test_norm = x_test.astype('float32') / 255.0

print('최소값:',np.min(x_train_norm), ' 최대값:',np.max(x_train_norm))
#print(f"x_train.shape: {x_train.shape}")

최소값: 0  최대값: 255
최소값: 0.0  최대값: 1.0


In [4]:
# Normalization - y (One-hot-encoding)
y_train = tf.keras.utils.to_categorical(y_train, 10)
y_test = tf.keras.utils.to_categorical(y_test, 10)

# Model

In [5]:
from keras.layers import Input,Conv2D,Dropout, Dense,Flatten,GlobalAveragePooling2D,MaxPool2D
from keras.models import Model

#  tf.keras의 Sequential API를 이용하여 LeNet 모델 생성
model=keras.models.Sequential()
model.add(keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(28,28,1)))
model.add(keras.layers.MaxPool2D(2,2))
model.add(keras.layers.Conv2D(32, (3,3), activation='relu'))
model.add(keras.layers.MaxPooling2D((2,2)))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(32, activation='relu'))
model.add(keras.layers.Dense(10, activation='softmax'))


In [6]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 16)        160       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 16)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 32)        4640      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 32)          0         
_________________________________________________________________
flatten (Flatten)            (None, 800)               0         
_________________________________________________________________
dense (Dense)                (None, 32)                25632     
_________________________________________________________________
dense_1 (Dense)              (None, 10)                3

In [7]:
# 채널수 맞추기
print("Before Reshape - x_train_norm shape: {}".format(x_train_norm.shape))
print("Before Reshape - x_test_norm shape: {}".format(x_test_norm.shape))

x_train_reshaped=x_train_norm.reshape( -1, 28, 28, 1)  # 데이터갯수에 -1을 쓰면 reshape시 자동계산됩니다.
x_test_reshaped=x_test_norm.reshape( -1, 28, 28, 1)

print("After Reshape - x_train_reshaped shape: {}".format(x_train_reshaped.shape))
print("After Reshape - x_test_reshaped shape: {}".format(x_test_reshaped.shape))


Before Reshape - x_train_norm shape: (60000, 28, 28)
Before Reshape - x_test_norm shape: (10000, 28, 28)
After Reshape - x_train_reshaped shape: (60000, 28, 28, 1)
After Reshape - x_test_reshaped shape: (10000, 28, 28, 1)


# Train Model

In [8]:
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=['accuracy'])


In [9]:
%%time 

model.fit(x_train_reshaped, y_train, epochs=10)

Train on 60000 samples
Epoch 1/10
60000/60000 [==============================] - 6s 101us/sample - loss: 0.2038 - accuracy: 0.9390
Epoch 2/10
60000/60000 [==============================] - 4s 64us/sample - loss: 0.0695 - accuracy: 0.9789
Epoch 3/10
60000/60000 [==============================] - 4s 65us/sample - loss: 0.0497 - accuracy: 0.9846
Epoch 4/10
60000/60000 [==============================] - 4s 66us/sample - loss: 0.0376 - accuracy: 0.9885
Epoch 5/10
60000/60000 [==============================] - 4s 59us/sample - loss: 0.0299 - accuracy: 0.9905
Epoch 6/10
60000/60000 [==============================] - 3s 57us/sample - loss: 0.0250 - accuracy: 0.9920
Epoch 7/10
60000/60000 [==============================] - 4s 60us/sample - loss: 0.0203 - accuracy: 0.9935
Epoch 8/10
60000/60000 [==============================] - 4s 70us/sample - loss: 0.0162 - accuracy: 0.9942
Epoch 9/10
60000/60000 [==============================] - 4s 60us/sample - loss: 0.0146 - accuracy: 0.9951
Epoch 10/10
6

# Model Evaluation

In [10]:
predictions = model.predict(x_test_reshaped)
accuracy = np.sum(np.argmax(predictions, axis=1) == np.argmax(y_test, axis=1)) / len(y_test)
print('Accuracy on benign test example: {}%'.format(accuracy*100))

/home/aiffel/anaconda3/envs/aiffel2/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:2325: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


Accuracy on benign test example: 98.65%


# Add noise

In [11]:
# noise 3
noise3 = np.random.normal(0,0.05,x_test.shape)
x_test_noise3 = x_test_norm + noise3
x_test_noise3_reshape = x_test_noise3.reshape(-1,28,28,1)

In [12]:
# 1차
noise = np.random.normal(0,0.05,x_test.shape)
noise_fgsm = x_test_norm + noise

for i in range (5):
    new_noise = np.random.normal(0,0.05,x_test.shape)
    x_test_noise = x_test_norm + new_noise
    noise_fgsm = np.concatenate((noise_fgsm,x_test_noise),axis=0)
    
noise_fgsm_reshape = noise_fgsm.reshape(-1,28,28,1)

In [13]:
noise_fgsm_reshape.shape

(60000, 28, 28, 1)

# 1. Projected Gradient Descent (PGD)

# Adversarial Examples

In [14]:
# ART
import art
from art.estimators.classification import KerasClassifier

#Wrap model with ART Keras Classifier
estimator = KerasClassifier(model=model, clip_values=(0,1), use_logits=True)

## 1-1) PGD_origin
### 원본 이미지 x_test

In [14]:
%%time 

from art.attacks.evasion import ProjectedGradientDescent
#Create PGD Instance
attack_pgd = ProjectedGradientDescent(estimator, eps=0.1,
                                     batch_size=100)

#Create Adversarial Examples
examples_pgd = attack_pgd.generate(x_test_reshaped,y_test)

PGD - Random Initializations:   0%|          | 0/1 [00:00<?, ?it/s]

PGD - Iterations:   0%|          | 0/100 [00:00<?, ?it/s]

PGD - Random Initializations:   0%|          | 0/1 [00:00<?, ?it/s]

PGD - Iterations:   0%|          | 0/100 [00:00<?, ?it/s]

PGD - Random Initializations:   0%|          | 0/1 [00:00<?, ?it/s]

...

PGD - Random Initializations:   0%|          | 0/1 [00:00<?, ?it/s]

PGD - Iterations:   0%|          | 0/100 [00:00<?, ?it/s]

PGD - Random Initializations:   0%|          | 0/1 [00:00<?, ?it/s]

PGD - Iterations:   0%|          | 0/100 [00:00<?, ?it/s]

CPU times: user 47.1 s, sys: 2.27 s, total: 49.3 s
Wall time: 42.7 s


In [46]:
prediction = model.predict(examples_pgd)

accuracy = np.sum(np.argmax(prediction, axis=1) == np.argmax(y_test, axis=1)) / len(y_test)
print('Accuracy on Adversarial Exmaples: {}%'.format(accuracy * 100))

Accuracy on Adversarial Exmaples: 63.23%


# PGD + noise image

In [57]:
# noise 3
noise3 = np.random.normal(0,0.05,x_test.shape)
x_test_noise3 = x_test_norm + noise3
x_test_noise3_reshape = x_test_noise3.reshape(-1,28,28,1)

In [58]:
%%time 

from art.attacks.evasion import ProjectedGradientDescent
#Create PGD Instance
attack_pgd = ProjectedGradientDescent(estimator, eps=0.1,
                                     batch_size=100)

#Create Adversarial Examples
examples_pgd = attack_pgd.generate(x_test_noise3_reshape,y_test)

PGD - Random Initializations:   0%|          | 0/1 [00:00<?, ?it/s]

PGD - Iterations:   0%|          | 0/100 [00:00<?, ?it/s]

PGD - Random Initializations:   0%|          | 0/1 [00:00<?, ?it/s]

PGD - Iterations:   0%|          | 0/100 [00:00<?, ?it/s]

PGD - Random Initializations:   0%|          | 0/1 [00:00<?, ?it/s]

...

PGD - Random Initializations:   0%|          | 0/1 [00:00<?, ?it/s]

PGD - Iterations:   0%|          | 0/100 [00:00<?, ?it/s]

PGD - Random Initializations:   0%|          | 0/1 [00:00<?, ?it/s]

PGD - Iterations:   0%|          | 0/100 [00:00<?, ?it/s]

CPU times: user 50.5 s, sys: 1.82 s, total: 52.3 s
Wall time: 44.8 s


In [59]:
prediction = model.predict(examples_pgd)

accuracy = np.sum(np.argmax(prediction, axis=1) == np.argmax(y_test, axis=1)) / len(y_test)
print('Accuracy on Adversarial Exmaples: {}%'.format(accuracy * 100))

Accuracy on Adversarial Exmaples: 51.09%


In [60]:
# noise 3
noise3 = np.random.normal(0,0.05,x_test.shape)
x_test_noise3 = x_test_norm + noise3
x_test_noise3_reshape = x_test_noise3.reshape(-1,28,28,1)

In [61]:
# noise 3
noise3_1 = np.random.normal(0,0.05,x_test.shape)
x_test_noise3_1 = x_test_norm + noise3_1
x_test_noise3_1_reshape = x_test_noise3_1.reshape(-1,28,28,1)

In [62]:
add_x_test = np.concatenate((x_test_noise3,x_test_noise3_1),axis=0)
add_x_test.shape

(20000, 28, 28)

In [63]:
add_x_test_reshape = add_x_test.reshape(-1,28,28,1)
add_x_test_reshape.shape

(20000, 28, 28, 1)

In [66]:
y_test2 = y_test.copy()

In [67]:
y_test_add = np.concatenate((y_test,y_test2),axis=0)
y_test_add.shape

(20000, 10)

In [68]:
%%time 

from art.attacks.evasion import ProjectedGradientDescent
#Create PGD Instance
attack_pgd = ProjectedGradientDescent(estimator, eps=0.1,
                                     batch_size=100)

#Create Adversarial Examples
examples_pgd = attack_pgd.generate(add_x_test_reshape,y_test_add)

PGD - Random Initializations:   0%|          | 0/1 [00:00<?, ?it/s]

PGD - Iterations:   0%|          | 0/100 [00:00<?, ?it/s]

PGD - Random Initializations:   0%|          | 0/1 [00:00<?, ?it/s]

PGD - Iterations:   0%|          | 0/100 [00:00<?, ?it/s]

PGD - Random Initializations:   0%|          | 0/1 [00:00<?, ?it/s]

...

PGD - Random Initializations:   0%|          | 0/1 [00:00<?, ?it/s]

PGD - Iterations:   0%|          | 0/100 [00:00<?, ?it/s]

PGD - Random Initializations:   0%|          | 0/1 [00:00<?, ?it/s]

PGD - Iterations:   0%|          | 0/100 [00:00<?, ?it/s]

CPU times: user 1min 38s, sys: 3.37 s, total: 1min 41s
Wall time: 1min 26s


In [69]:
prediction = model.predict(examples_pgd)

accuracy = np.sum(np.argmax(prediction, axis=1) == np.argmax(y_test_add, axis=1)) / len(y_test_add)
print('Accuracy on Adversarial Exmaples: {}%'.format(accuracy * 100))

Accuracy on Adversarial Exmaples: 51.11%


## 1-1) FGSM_origin
### 원본 이미지 x_test


fgsm 이미지 갯수 늘이기


In [76]:
%%time 

from art.attacks.evasion import FastGradientMethod
#Create FGSM Instance
attack_fgsm = FastGradientMethod(estimator, eps=0.1)

#Create Adversarial Examples
examples_fgsm = attack_fgsm.generate(x_test_reshaped,y_test)

CPU times: user 1.41 s, sys: 102 ms, total: 1.51 s
Wall time: 1.07 s


In [77]:
prediction_fgsm = model.predict(examples_fgsm)

acc_fgsm = np.sum(np.argmax(prediction_fgsm, axis=1) == np.argmax(y_test, axis=1)) / len(y_test)
print('Accuracy on Adversarial Exmaples: {}%'.format(acc_fgsm * 100))

Accuracy on Adversarial Exmaples: 75.61%


# FGSM + noise image

In [78]:
# noise 3
noise3 = np.random.normal(0,0.05,x_test.shape)
x_test_noise3 = x_test_norm + noise3
x_test_noise3_reshape = x_test_noise3.reshape(-1,28,28,1)

In [79]:
%%time 

from art.attacks.evasion import FastGradientMethod
#Create FGSM Instance
attack_fgsm = FastGradientMethod(estimator, eps=0.1)

#Create Adversarial Examples
examples_fgsm = attack_fgsm.generate(x_test_noise3_reshape,y_test)

CPU times: user 1.26 s, sys: 136 ms, total: 1.39 s
Wall time: 1.06 s


In [81]:
prediction_fgsm = model.predict(examples_fgsm)

acc_fgsm = np.sum(np.argmax(prediction_fgsm, axis=1) == np.argmax(y_test, axis=1)) / len(y_test)
print('Accuracy on Adversarial Exmaples: {}%'.format(acc_fgsm * 100))

Accuracy on Adversarial Exmaples: 70.00999999999999%


# FGSM +noise image * 34회

In [87]:
# 1차
noise = np.random.normal(0,0.05,x_test.shape)
noise_fgsm = x_test_norm + noise

for i in range (33):
    new_noise = np.random.normal(0,0.05,x_test.shape)
    x_test_noise = x_test_norm + new_noise
    noise_fgsm = np.concatenate((noise_fgsm,x_test_noise),axis=0)
    
noise_fgsm_reshape = noise_fgsm.reshape(-1,28,28,1)

In [88]:
noise_fgsm_reshape.shape

(340000, 28, 28, 1)

In [104]:
y_test2 = y_test.copy()
y_test_add = np.concatenate((y_test,y_test2),axis=0)

for i in range (32):
    y_test2 = y_test.copy()
    y_test_add = np.concatenate((y_test_add,y_test2),axis=0)


In [105]:
y_test_add.shape

(340000, 10)

In [106]:
%%time 

from art.attacks.evasion import FastGradientMethod
#Create FGSM Instance
attack_fgsm = FastGradientMethod(estimator, eps=0.1)

#Create Adversarial Examples
examples_fgsm = attack_fgsm.generate(noise_fgsm_reshape,y_test_add)

CPU times: user 48.8 s, sys: 4.76 s, total: 53.6 s
Wall time: 39.4 s


In [107]:
prediction_fgsm = model.predict(examples_fgsm)

acc_fgsm = np.sum(np.argmax(prediction_fgsm, axis=1) == np.argmax(y_test_add, axis=1)) / len(y_test_add)
print('Accuracy on Adversarial Exmaples: {}%'.format(acc_fgsm * 100))

Accuracy on Adversarial Exmaples: 70.0864705882353%
